# Test of Network architecture in the 1D case

We test different numbers of hidden layers and nodes per layers and analyze which architecture leads to the best results. Here, only the one-dimensional heat equation is considered. We expect that the findings can be more or less used in the two-dimensional case as well.

In [1]:
import torch
import numpy as np
from pinn import heat_nn
import matplotlib.pyplot as plt
import time

We consider the one-dimensional heat equation
$$
\partial_t u - \kappa \Delta u = f
$$
where $\kappa = 0.1$ and $f(x) = \sin (\pi x)$.

In [2]:
### PDE parameters

# dimension
dim=1

# diffusion coefficient
kappa = 0.1

# initial condition
u_0 = lambda x: torch.sin(torch.pi * x) + torch.sin(4 * torch.pi * x)

# right hand side of the equation
rhs = lambda x, t: torch.sin(torch.pi * x)

# analytic solution
u_analytic = lambda x, t: (1 - 1 / (0.1 * torch.pi**2)) * torch.sin(torch.pi * x) * torch.exp(-torch.pi**2 * 0.1 * t
                    ) + torch.sin(4 * torch.pi * x) * torch.exp(- 16 * torch.pi**2 * 0.1 * t
                    ) + 1 / (0.1 * torch.pi**2) * torch.sin(torch.pi * x)

Test different numbers of layers and different sizes.

In [6]:
### create a list of different layer combinations
### output layer has always size 1

# list of layers to test
layers_list = [[32, 64, 1],
               [64, 64, 1],
               [32, 64, 64, 1],
               [64, 64, 32, 1],
               [64, 32, 1],
               [128, 64, 32, 1],
               [128, 64, 1],
               [128, 128, 64, 64, 32, 1],
               [128, 128, 64, 32, 1],
               [32, 64, 128, 128, 1],
               [128, 128, 1],
               [128, 128, 128, 1],
               [512, 1],
               [256, 256, 1],
               [128, 128, 128, 128, 1]]

for layers in layers_list:

    # use a random seed for comparability
    np.random.seed(238)
    torch.manual_seed(301)

    # measure computation time
    start = time.perf_counter()

    activations = [torch.tanh]*(len(layers)-1) + [None]
    pde_nn = heat_nn(layers, activations, dim, u_0, kappa, rhs, reg=0)
    pde_nn.set_analytic_solution(u_analytic)
    N_colloc = 100
    pde_nn.set_data(N_colloc)

    # gives relatively good results (compared to other parameters, still bad though)
    pde_nn.train(lr=1e-2, weight_decay=0.0, epochs = 400, opt_time_scale =True, print_epochs=50)
    # LBFGS needs approximately 100 epochs, 30 iterations for kappa = 1
    # if kappa = 0.1, better choose more iterations, less epochs
    pde_nn.train_lbfgs(lr=1, opt_time_scale = True, epochs=10, max_iter=50)

    end = time.perf_counter()

    ### error measured in L^2 and L^{\infty} norm
    L_2_err = pde_nn.L_2_error()
    L_infty_err = pde_nn.L_infty_error()
    print("Layers:            ", layers)
    print("L^2 error:         ", L_2_err)
    print("L_^{infty}_error:  ", L_infty_err)
    print("Runtime:           ", end - start, "\n")

Epoch 0, Loss: 47.422729, MSE: 0.216549
Epoch 50, Loss: 23.440220, MSE: 0.182161
Epoch 100, Loss: 13.175997, MSE: 0.088233
Epoch 150, Loss: 9.694346, MSE: 0.071735
Epoch 200, Loss: 7.817989, MSE: 0.053309
Epoch 250, Loss: 6.203895, MSE: 0.035725
Epoch 300, Loss: 5.280070, MSE: 0.027284
Epoch 350, Loss: 4.721897, MSE: 0.023851
Epoch 399, Loss: 4.097630, MSE: 0.021244
Epoch 0, Loss: 1.884087, MSE: 0.003763
Epoch 1, Loss: 0.823469, MSE: 0.000954
Epoch 2, Loss: 0.421720, MSE: 0.000261
Epoch 3, Loss: 0.242479, MSE: 0.000193
Epoch 4, Loss: 0.148513, MSE: 0.000118
Epoch 5, Loss: 0.095363, MSE: 0.000047
Epoch 6, Loss: 0.064038, MSE: 0.000024
Epoch 7, Loss: 0.046681, MSE: 0.000020
Epoch 8, Loss: 0.036265, MSE: 0.000013
Epoch 9, Loss: 0.025542, MSE: 0.000010
Layers:             [32, 64, 1]
L^2 error:          0.00139357
L_^{infty}_error:   0.013494074
Runtime:            154.4992936310009 

Epoch 0, Loss: 47.525280, MSE: 0.201070
Epoch 50, Loss: 21.297403, MSE: 0.154924
Epoch 100, Loss: 15.61219

KeyboardInterrupt: 